In [ ]:

import pandas as pd
import re
import os

# Get a list of all .csv files in the current directory
csv_files = [f for f in os.listdir('.') if f.endswith('.csv')]
#print(csv_files)
for csv_file in csv_files:
  if 'respuestas_' in csv_file:
    # Extract the part after "respuestas_"
    model_name = csv_file.split('respuestas_')[1].split('.')[0]
    # Extract the first 5 letters
    model_name_short = model_name[:5]
    # Extract the first number
    match = re.search(r'\d+', model_name)
    if match:
      model_number = match.group()
    else:
      model_number = ''
    # Create the DataFrame name
    df_name = f"df{model_name_short}{model_number}"

    # Read the CSV file into a DataFrame
    globals()[df_name] = pd.read_csv(csv_file)
    print(f"Created DataFrame: {df_name}")

# Access the created DataFrames like this:
# print(dfMini1.head()) # Example for a DataFrame named dfMini1


Created DataFrame: dfMistr24
Created DataFrame: dfsalam7


In [ ]:

import re

def starts_with_si(text):
    """Checks if a string starts with 'si' (ignoring case and accents).
    Returns 1 if it starts with 'si' or 'sí', 0 if it starts with 'no', and 2 otherwise.
    """
    if isinstance(text, str):
        text = text.lower()
        text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
        if text.startswith('si') or text.startswith('sí') or text.startswith('sim') or text.startswith('bai'):
            return 1
        elif text.startswith('no') or text.startswith('não') or text.startswith('nao') or text.startswith('ez'):
            return 0
        else:
            return 2
    else:
        return 2  # Return 2 for non-string inputs

# Example using dfMinis3
#dfMinis8['starts_with_si'] = dfMinis8['Respuesta del Modelo'].apply(starts_with_si)

# View the updated DataFrame
#print(dfMinis8.head())


In [ ]:

#dfs_to_process = [dfsalam2, dfMinis3, dfDeepS1, dfLlama3, dfQwen22, dfMistr7, dfsalam7, dfgemma2, dfMinis8]
dfs_to_process = [dfMistr24, dfsalam7]
#dfs_to_process = [dfLatxa3]

for df in dfs_to_process:
  if 'Respuesta del Modelo' in df.columns:
    df['starts_with_si'] = df['Respuesta del Modelo'].apply(starts_with_si)


In [ ]:
dfTop=pd.read_csv("sample_df.csv")

In [ ]:
#dfs_to_process = [dfsalam2, dfMinis3, dfDeepS1, dfLlama3, dfQwen22, dfMistr7, dfsalam7, dfgemma2, dfMinis8]
dfs_to_process = [dfMistr24, dfsalam7]
#dfs_to_process = [dfLatxa3]

for df in dfs_to_process:
  if 'IS_IRONIC' in dfTop.columns:
    df['original_label'] = dfTop['IS_IRONIC']
  else:
    print("Warning: 'IS_IRONIC' column not found in dfTop.")


In [ ]:
# Acc

#dfs_to_process = [dfsalam2, dfMinis3, dfDeepS1, dfLlama3, dfQwen22, dfMistr7, dfsalam7, dfgemma2, dfMinis8]
#df_names = ['dfsalam2', 'dfMinis3', 'dfDeepS1', 'dfLlama3', 'dfQwen22', 'dfMistr7', 'dfsalam7', 'dfgemma2', 'dfMinis8']
dfs_to_process = [dfMistr24, dfsalam7]
df_names = ["dfMistr24", "dfsalam7"]
#dfs_to_process = [dfLatxa3]
#df_names = ["dfLatxa3"]


for df, df_name in zip(dfs_to_process, df_names):
  if 'original_label' in df.columns and 'starts_with_si' in df.columns:
    percentage_equal = (df['original_label'] == df['starts_with_si']).mean() * 100
    print(f"{df_name}: Percentage of equal values between original_label and model prediction: {percentage_equal:.2f}%")


dfMistr24: Percentage of equal values between original_label and model prediction: 45.80%
dfsalam7: Percentage of equal values between original_label and model prediction: 43.20%


In [ ]:
# f1 score based on the existence of only 0 and 1 labels (in the df[starts_with_si] there is values 0,1 and 2 but 2 only represent a wrong answer)

#dfs_to_process = [dfsalam2, dfMinis3, dfDeepS1, dfLlama3, dfQwen22, dfMistr7, dfsalam7, dfgemma2, dfMinis8]
#df_names = ['dfsalam2', 'dfMinis3', 'dfDeepS1', 'dfLlama3', 'dfQwen22', 'dfMistr7', 'dfsalam7', 'dfgemma2', 'dfMinis8']

dfs_to_process = [dfMistr24, dfsalam7]
df_names = ["dfMistr24", "dfsalam7"]
#dfs_to_process = [dfLatxa3]
#df_names = ["dfLatxa3"]
for df, df_name in zip(dfs_to_process, df_names):
  if 'original_label' in df.columns and 'starts_with_si' in df.columns:
    # Create a copy of the 'starts_with_si' column
    df['adjusted_starts_with_si'] = df['starts_with_si'].copy()

    # Iterate through the rows and adjust 'starts_with_si' based on 'original_label'
    for index, row in df.iterrows():
      if row['starts_with_si'] == 2:  # Only modify rows where it's 2 (meaning neither 'si' nor 'no')
        if row['original_label'] == 0:  # If correct label is 'no' (0)
          df.loc[index, 'adjusted_starts_with_si'] = 1  # Change predicted label to 'si' (1)
        elif row['original_label'] == 1:  # If correct label is 'si' (1)
          df.loc[index, 'adjusted_starts_with_si'] = 0  # Change predicted label to 'no' (0)

    # Ensure that 'original_label' and 'adjusted_starts_with_si' have the same length
    min_len = min(len(df['original_label']), len(df['adjusted_starts_with_si']))

    # Calculate the F1-score using only 0 and 1 labels
    f1 = f1_score(df['original_label'].values[:min_len], df['adjusted_starts_with_si'].values[:min_len], average='binary')
    print(f"{df_name}: F1-score (with adjusted labels): {f1:.3f}")


dfMistr24: F1-score (with adjusted labels): 0.523
dfsalam7: F1-score (with adjusted labels): 0.380
